In [20]:
import pandas as pd
import unicodedata


In [21]:
df = pd.read_csv('../data/data.csv', encoding='utf-8')
df

,ElementId,Value
0,122430,Video: https://youtu.be/z-IwPw5gwD8 Fecha máxi...
1,69562,1. NEW. Se habilitó el Log para registrar las ...
2,106534,29.6-FIX-01: Se realizó un ajuste en el mecani...
3,52821,1. New: Se implementó el sistema de autenticac...
4,110816,30.0-FIX-01: (HRP) Se realizaron varios ajuste...
...,...,...
146,96955,R29.1-FIX-01: FIX. (EG) Se corrigió el problem...
147,97580,R29.2-FT-01: NEW. Se reescribió todo el mecan...
148,98276,R29.3-FT-01: NEW. Se creó un mecanismo para cu...
149,115966,Video: 30.2-FT-01. (HR) Ahora es posible ut...


In [22]:
df['Value'] = df['Value'].str.replace('\xa0', ' ')

In [ ]:
import pandas as pd
import re

def split_text_by_regex(row):
    # Usar una expresión regular para dividir el texto por saltos de línea o cualquier otro patrón deseado
    # \s* captura cualquier cantidad de espacios en blanco (incluidos los saltos de línea) alrededor del separador
    # Puedes ajustar la expresión regular según tus necesidades
    fragments = re.split(r'\n\s*\n', row['Texto'], flags=re.IGNORECASE)
    # Filtrar líneas vacías o con solo espacios, y quitar espacios adicionales
    fragments = [line.strip() for line in fragments if line.strip()]
    row['Texto'] = fragments
    return row




In [ ]:
# Aplicar la función y expandir el DataFrame
df = df.apply(split_text_by_regex, axis=1)
df = df.explode('Value').reset_index(drop=True)
df

In [4]:
from langchain.document_loaders import DataFrameLoader

loader = DataFrameLoader(df, page_content_column="Value")
data = loader.load()

In [5]:
from pprint import pprint

pprint(data[:5])

[Document(page_content='Video: https://youtu.be/z-IwPw5gwD8 Fecha máxima para comentarios y correcciones: Marzo 21, 2024   30.4-FIX-01 (CG)  Ajustado error cuando se usaban suggest en el filtro no traía resultadosPruebas y aplicabilidad: Makro y dónde aplique, se debe probar que al usar en el filtro campos suggest, este retorne los resultados que corresponden al filtro aplicado. 30.4-FIX-02 (CG)  Ajustado la funcionalidad del parametro VersionChangesMetaDataIdPruebas y aplicabilidad: Ebsa y dónde aplique. 30.4-FIX-03 (CG) Ajustado error que se presentaba al enviar las notificaciones para responder encuestas (SURVEYS II). Anteriormente, existía un límite de 12 horas para abrir la encuesta a través del enlace enviado por correo electrónico, después del límite se generaba un error. Pruebas y aplicabilidad: Seguros Mundial. Probar generando encuentas y poder usar el link para responderla despues de doce horas. 30.4-FT-01 (CG) Se ha implementado una nueva funcionalidad que permite definir u

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    length_function=len,
    chunk_overlap=100
)

documents = text_splitter.split_documents(data)

In [7]:
len(documents)

1665

In [8]:
documents[0]

Document(page_content='Video: https://youtu.be/z-IwPw5gwD8 Fecha máxima para comentarios y correcciones: Marzo 21, 2024   30.4-FIX-01 (CG)  Ajustado error cuando se usaban suggest en el filtro no traía resultadosPruebas y aplicabilidad: Makro y dónde aplique, se debe probar que al usar en el filtro campos suggest, este retorne los resultados que corresponden al filtro aplicado. 30.4-FIX-02 (CG)  Ajustado la funcionalidad del parametro VersionChangesMetaDataIdPruebas y aplicabilidad: Ebsa y dónde aplique. 30.4-FIX-03', metadata={'ElementId': 122430})

In [9]:
documents[1]

Document(page_content='del parametro VersionChangesMetaDataIdPruebas y aplicabilidad: Ebsa y dónde aplique. 30.4-FIX-03 (CG) Ajustado error que se presentaba al enviar las notificaciones para responder encuestas (SURVEYS II). Anteriormente, existía un límite de 12 horas para abrir la encuesta a través del enlace enviado por correo electrónico, después del límite se generaba un error. Pruebas y aplicabilidad: Seguros Mundial. Probar generando encuentas y poder usar el link para responderla despues de doce horas.', metadata={'ElementId': 122430})

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings_st = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
## Otro modelo en español que podríamos usar es "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"

/home/garceseder2/chatbot/dev_chat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from langchain.vectorstores import Chroma
NOMBRE_INDICE_CHROMA = "../databases/embeddings-public"

In [12]:

vectorstore_chroma = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_st,
    persist_directory=NOMBRE_INDICE_CHROMA
)

In [13]:
vectorstore_chroma.persist()

In [14]:
vectorstore_chroma = Chroma(
    persist_directory=NOMBRE_INDICE_CHROMA,
    embedding_function=embeddings_st
)

In [18]:
query = "Cual es el parametro para configurar el nuevo Pivot?"

docs = vectorstore_chroma.similarity_search_with_score(query, k=10)

In [19]:
docs

[(Document(page_content='en cuenta de forma correcta el órden en que se establecen (SENSE), arrastrando y soltando en el formulario. 13. FIX: Se ajustó el mecanismo de creación automática de embebidos para dejarlos con la versión inicial establecida por el parámetro InitialVersion. 14. FIX: Se ajustó el mecanismo de creación automática de embebidos, para ejecutar los calculos de metadatas calculadas e infotext. 15. FIX:  (Metadata no trae responsables de los permisos configurados). 16. FIX:  17. FIX: [Breaking Change]', metadata={'ElementId': 47716}),
  8.122855186462402),
 (Document(page_content='de obtener permisos no editables, responsables de flujo durante el cambio de estado. Pruebas y Aplicabilidad: Donde sea requerido. Probar configuración de elementos durante cambios de estado, asignación correcta de los permisos y responsables en el flujo    29.4-FIX-05. Corregido un problema de desempeño relacionado con el mecanismo que obtiene información del nombre real de un anexo, cuando 